In [1]:
import tensorflow as tf
import pickle

In [69]:
f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

In [3]:
mode = 'train'

In [4]:
# Parameters

batch_size = 50
vocab_size = len(vocab)
unk_vocab_size = 1000
word_emb_size = len(word_embedding[0])
max_sent_size = 100
hidden_size = 100
learning_rate = 0.001
keep_prob = 0.5
input_keep_prob = 0.7

In [5]:

x = tf.placeholder('int32', [batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
x_mask  = tf.placeholder('bool', [batch_size, max_sent_size], name='x_mask') 

is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')


In [6]:
input_keep_prob = tf.cond(is_train,lambda:input_keep_prob, lambda:tf.constant(1.0))


In [7]:
with tf.name_scope("word_embedding"):

    if mode == 'train':
        unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))
    else:
        unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')


    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)

    Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)  

In [8]:
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)

In [9]:
cell_fw = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
cell_bw = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

d_cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob=input_keep_prob)
d_cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob=input_keep_prob)

In [10]:
(fw_h, bw_h), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw, d_cell_bw, Wx, sequence_length=x_len, dtype='float', scope='LSTM')


In [11]:
h = tf.concat([fw_h, bw_h], 2)

In [67]:
h_masked = tf.expand_dims(tf.boolean_mask(h[0], x_mask[0]), 0)
for i in range(1, batch_size):
    h_masked = tf.concat([h_masked, tf.expand_dims(tf.boolean_mask(h[i], x_mask[i]), 0)], 0)

In [68]:
h_masked

<tf.Tensor 'concat_101:0' shape=(50, ?, 200) dtype=float32>

In [49]:
sess = tf.Session()

In [50]:
sess.run(tf.global_variables_initializer())